# Inferencing with included SpaceNet models and the `solaris` Python API

We've included a number of SpaceNet models with `solaris`, including pre-trained model weights. You can find more information about your model choices [here](../pretrained_models.html) and the original competitors' code for the models [here](https://github.com/spacenetchallenge/spacenet_off_nadir_solutions).

For this tutorial we'll walk through running inference with XD_XD's SpaceNet 4 model. We'll use the config file for that model, which you can find [here](https://github.com/CosmiQ/solaris/blob/master/solaris/nets/configs/xdxd_spacenet4.yml).

You'll also need to [create the image reference files](creating_im_reference_csvs.ipynb) before you start.

First, we'll load the configuration:

In [2]:
import solaris as sol

config = sol.utils.config.parse('/Users/nweir/code/cosmiq_repos/solaris/solaris/nets/configs/xdxd_spacenet4.yml')
config

{'model_name': 'xdxd_spacenet4',
 'model_path': None,
 'train': False,
 'infer': True,
 'pretrained': True,
 'nn_framework': 'torch',
 'batch_size': 12,
 'data_specs': {'width': 512,
  'height': 512,
  'image_type': 'zscore',
  'rescale': False,
  'rescale_minima': 'auto',
  'rescale_maxima': 'auto',
  'channels': 4,
  'label_type': 'mask',
  'is_categorical': False,
  'mask_channels': 1,
  'val_holdout_frac': 0.2,
  'data_workers': None},
 'training_data_csv': '/path/to/training_df.csv',
 'validation_data_csv': None,
 'inference_data_csv': '/path/to/test_df.csv',
 'training_augmentation': {'augmentations': {'DropChannel': {'idx': 3,
    'axis': 2},
   'HorizontalFlip': {'p': 0.5},
   'RandomRotate90': {'p': 0.5},
   'RandomCrop': {'height': 512, 'width': 512, 'p': 1.0},
   'Normalize': {'mean': [0.006479, 0.009328, 0.01123],
    'std': [0.004986, 0.004964, 0.00495],
    'max_pixel_value': 65535.0,
    'p': 1.0}},
  'p': 1.0,
  'shuffle': True},
 'validation_augmentation': {'augmentati

As you can see, the YAML gets parsed into a set of nested dictionaries by `solaris`. Relevant pieces of that config then get read during training.

Inferencing is _very_ similar to [training](api_training_spacenet.ipynb), with one major difference: you'll load in and pass the reference CSV for your inference dataset as an argument to the `Inferer` object. `solaris` is set up this way so that you can quickly and easily iterate through inference on a number of inputs without having to re-instantiate your inferencer each time.

In [ ]:
inferer = sol.nets.infer.Inferer(config)
inference_df = sol.nets.infer.get_infer_df(config)
inferer(inference_df)

The above commands will create prediction segmentation masks for each input image in the `output_dir` specified in your `config`. You can then [use sol.vector.mask.mask_to_poly_geojson to convert these predicted masks to vector-formatted geometries](api_mask_to_vector.ipynb).